## utils.py

## ddpm.py

In [34]:
noise_steps=10 #1000
beta_start=1e-2 #4
beta_end=0.2 #0.02
img_size=256
device="cpu"

def prepare_noise_schedule():
    return torch.linspace(beta_start, beta_end, noise_steps)

beta = prepare_noise_schedule().to(device)
alpha = 1. - beta
alpha_hat = torch.cumprod(alpha, dim=0)

In [36]:
beta

tensor([0.0100, 0.0311, 0.0522, 0.0733, 0.0944, 0.1156, 0.1367, 0.1578, 0.1789,
        0.2000])

In [38]:
alpha

tensor([0.9900, 0.9689, 0.9478, 0.9267, 0.9056, 0.8844, 0.8633, 0.8422, 0.8211,
        0.8000])

In [40]:
alpha_hat

tensor([0.9900, 0.9592, 0.9091, 0.8424, 0.7629, 0.6747, 0.5825, 0.4906, 0.4028,
        0.3223])

In [50]:
t = 7
torch.sqrt(alpha_hat[t])

tensor(0.7004)

In [56]:
import torch

# Create a sample tensor
alpha_hat = torch.tensor([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])

# Extract the slice at index 1
alpha_hat_t = alpha_hat[1]  # Tensor of shape (3,)
print(alpha_hat_t)
# Calculate the square root
sqrt_alpha_hat_t = torch.sqrt(alpha_hat_t)  # Tensor of shape (3,)
print(sqrt_alpha_hat_t)
# Expand the tensor to have additional dimensions
expanded_sqrt_alpha_hat_t = sqrt_alpha_hat_t[:, None, None, None]  # Tensor of shape (3, 1, 1, 1)

print(expanded_sqrt_alpha_hat_t)

tensor([4., 5., 6.])
tensor([2.0000, 2.2361, 2.4495])
tensor([[[[2.0000]]],


        [[[2.2361]]],


        [[[2.4495]]]])


In [66]:
print(expanded_sqrt_alpha_hat_t[2][0][0][0])
alpha_hat.shape

tensor(2.4495)


torch.Size([2, 3])

In [7]:
import os
import torch
import torch.nn as nn
from matplotlib import pyplot as plt
from tqdm import tqdm
from torch import optim
#from utils import *
#from modules import UNet
import logging
from torch.utils.tensorboard import SummaryWriter #!pip install tensorboard

logging.basicConfig(format="%(asctime)s - %(levelname)s: %(message)s", level=logging.INFO, datefmt="%I:%M:%S")


class Diffusion:
    def __init__(self, noise_steps=1000, beta_start=1e-4, beta_end=0.02, img_size=256, device="cuda"):
        self.noise_steps = noise_steps
        self.beta_start = beta_start
        self.beta_end = beta_end
        self.img_size = img_size
        self.device = device

        self.beta = self.prepare_noise_schedule().to(device)
        self.alpha = 1. - self.beta
        self.alpha_hat = torch.cumprod(self.alpha, dim=0)

    def prepare_noise_schedule(self):
        return torch.linspace(self.beta_start, self.beta_end, self.noise_steps)

    def noise_images(self, x, t):
        sqrt_alpha_hat = torch.sqrt(self.alpha_hat[t])[:, None, None, None]
        print(sqrt_alpha_hat[0])
        sqrt_one_minus_alpha_hat = torch.sqrt(1 - self.alpha_hat[t])[:, None, None, None]
        Ɛ = torch.randn_like(x)
        return sqrt_alpha_hat * x + sqrt_one_minus_alpha_hat * Ɛ, Ɛ

    def sample_timesteps(self, n):
        return torch.randint(low=1, high=self.noise_steps, size=(n,))

    def sample(self, model, n):
        logging.info(f"Sampling {n} new images....")
        model.eval()
        with torch.no_grad():
            x = torch.randn((n, 3, self.img_size, self.img_size)).to(self.device)
            for i in tqdm(reversed(range(1, self.noise_steps)), position=0):
                t = (torch.ones(n) * i).long().to(self.device)
                predicted_noise = model(x, t)
                alpha = self.alpha[t][:, None, None, None]
                alpha_hat = self.alpha_hat[t][:, None, None, None]
                beta = self.beta[t][:, None, None, None]
                if i > 1:
                    noise = torch.randn_like(x)
                else:
                    noise = torch.zeros_like(x)
                x = 1 / torch.sqrt(alpha) * (x - ((1 - alpha) / (torch.sqrt(1 - alpha_hat))) * predicted_noise) + torch.sqrt(beta) * noise
        model.train()
        x = (x.clamp(-1, 1) + 1) / 2
        x = (x * 255).type(torch.uint8)
        return x



In [22]:
def train(args):
    setup_logging(args.run_name)
    device = args.device
    dataloader = get_data(args)
    model = UNet().to(device)
    optimizer = optim.AdamW(model.parameters(), lr=args.lr)
    mse = nn.MSELoss()
    diffusion = Diffusion(img_size=args.image_size, device=device)
    logger = SummaryWriter(os.path.join("runs", args.run_name))
    l = len(dataloader)

    for epoch in range(args.epochs):
        logging.info(f"Starting epoch {epoch}:")
        pbar = tqdm(dataloader)
        for i, (images, _) in enumerate(pbar):
            images = images.to(device)
            t = diffusion.sample_timesteps(images.shape[0]).to(device)
            x_t, noise = diffusion.noise_images(images, t)
            predicted_noise = model(x_t, t)
            loss = mse(noise, predicted_noise)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            pbar.set_postfix(MSE=loss.item())
            logger.add_scalar("MSE", loss.item(), global_step=epoch * l + i)

        sampled_images = diffusion.sample(model, n=images.shape[0])
        save_images(sampled_images, os.path.join("results", args.run_name, f"{epoch}.jpg"))
        torch.save(model.state_dict(), os.path.join("models", args.run_name, f"ckpt.pt"))



In [49]:

    args.device = torch.device("mps") if torch.backends.mps.is_available() else "cpu" #"cuda"
    # ckpt = torch.load("./working/orig/ckpt.pt")
    # model.load_state_dict(ckpt)

In [ ]:
if __name__ == '__main__':
    launch()
    # device = "cuda"
    # model = UNet().to(device)
    # ckpt = torch.load("./working/orig/ckpt.pt")
    # model.load_state_dict(ckpt)
    # diffusion = Diffusion(img_size=64, device=device)
    # x = diffusion.sample(model, 8)
    # print(x.shape)
    # plt.figure(figsize=(32, 32))
    # plt.imshow(torch.cat([
    #     torch.cat([i for i in x.cpu()], dim=-1),
    # ], dim=-2).permute(1, 2, 0).cpu())
    # plt.show()